# [IMAGPRO] Machine Problem 2: Affine Transformations
$\text{submitted by: Pierre Vincent C. Hernandez}$

### Prerequisites / Environment

In order to properly run this notebook and produce it's expected output the 
Jupyter Notebook must be running using:

- **Python**: `^3.10`

Here are the list of specific packages and their version that is going to 
be used in this notebook:

- **NumPy**: `1.26.3`
- **OpenCV**: `4.9.0.80`
- **Matplotlib**: `3.8.3`

### Imports

Import the necessary packages to run the notebook.

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline

## 1. Data Formatting

write text here

In [1]:
# write code here

## 2. Data Augmentation

write text here

In [ ]:
# write code here